# Neo4jによるKnowledge Graph (KG) Hands-On

In [1]:
# ! pip install --upgrade pip
# ! pip install neo4j
# ! pip install langchain_openai
# ! pip install dotenv

## 事前準備

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_openai import OpenAI
from neo4j import GraphDatabase, basic_auth
import json

### OpenAIのAPIキーおよびNeo4j認証関連の環境変数読み込み

In [5]:
load_dotenv(find_dotenv('../.env'))

True

### Neo4jへの接続

In [22]:
# Neo4jの接続情報
neo4j_url = os.environ["NEO4J_URI"]
neo4j_user = os.environ["NEO4J_USERNAME"]
neo4j_password = os.environ["NEO4J_PASSWORD"] 

print(f"Neo4j URL: {neo4j_url}")
print(f"Neo4j User: {neo4j_user}")
print(f"Neo4j Password: {neo4j_password}")

# Neo4jセッションの作成
driver = GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password))

Neo4j URL: bolt://localhost:7689
Neo4j User: neo4j
Neo4j Password: password


### Neo4jのクリーンナップ

In [23]:
def delete_all_data(tx):
    tx.run("MATCH (n) DETACH DELETE n")

with driver.session() as session:
    all_nodes = session.execute_write(delete_all_data)

print(all_nodes)

None


## LLMによるトリプル抽出関数

- LangChain + OpenAI を使って、文章から (主語, 関係, 目的語) のトリプルを抽出する関数
- トリプル（triple）とは、「主語（subject）・関係（relation）・目的語（object）」**の3つからなる知識の最小単位のこと



In [24]:
def extract_triples(text: str):
    llm = OpenAI(temperature=0.0)

    prompt = f"""# 課題
次の文章を読み、知識グラフにおける"エンティティ" と "エンティティ間の意味的な関係" を抽出し、  
以下の形式の JSON リストとして出力してください。

# 出力形式

- relation には、「は」「に」などの助詞を使わず、必ずトリプルの関係を示す言葉を使ってください。
- 例えば'出身', '職業', '提案', '受賞', '予測', '父', '子', '構成', '部品'など文脈に応じた意味的な関係をわかりやすい動詞や名詞で示してください
- relationは文章に直接明示されていない場合は、subjectとobjectのエンティティから推測してください
- 最終的に正しいrelationができているかどうか確認するために、そのrelationを使って文章を作り、元の文章と同じ意味になるかどうかを確認してください
- 出力時は全角の数値や特殊文字は半角に変換してください
- 改行やスペースを入れないようにしてください

```json
[
  {{
    "subject": "主語(固有名詞など)",
    "relation": "関係(例: '出身', '職業', '提案', '受賞', '予測', '父', '子', '構成', '部品' など)",
    "object": "目的語(固有名詞や物事の名称など)"
  }}
]
```

# 文章: {text}
"""

    response = llm.invoke(prompt)
    return json.loads(response)

## トリプル抽出のテスト

サンプル文章を使って、意味的なリレーションが抽出されるかを確認します。

In [25]:
test_text = """アルベルト・アインシュタインはドイツ生まれの理論物理学者であり、相対性理論を提唱した。1921年にはノーベル物理学賞を受賞した。"""
#test_text = """DeepXは、「あらゆる機械の自動化」を実現し、世界の生産現場で生じるさまざまな課題の解決を目指します。"""
triples_result = extract_triples(test_text)
print(triples_result)

[{'subject': 'アルベルト・アインシュタイン', 'relation': '出身', 'object': 'ドイツ'}, {'subject': 'アルベルト・アインシュタイン', 'relation': '職業', 'object': '理論物理学者'}, {'subject': 'アルベルト・アインシュタイン', 'relation': '提唱', 'object': '相対性理論'}, {'subject': 'アルベルト・アインシュタイン', 'relation': '受賞', 'object': 'ノーベル物理学賞'}, {'subject': 'ノーベル物理学賞', 'relation': '年', 'object': '1921'}]


## ✅ Neo4j 接続確認関数
この関数は、指定した接続情報を使って `RETURN 1` クエリを投げることで Neo4j に接続できるかを確認します。

In [26]:
# トリプルをCREATEする関数例
def create_kg_in_neo4j(triples):
    with driver.session() as session:
        for triple in triples:
            subject = triple["subject"]
            relation = triple["relation"]
            obj = triple["object"]
            
            # Cypherクエリ例:
            # MERGE でノードがなければ作成、あれば再利用し、リレーションも作成
            cypher = """
            MERGE (s:Entity {name: $subject})
            MERGE (o:Entity {name: $object})
            MERGE (s)-[r:RELATION {type: $relation}]->(o)
            """
            session.run(cypher, subject=subject, object=obj, relation=relation)

# 実行例
create_kg_in_neo4j(triples_result)